In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
import pandas as pd

df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

In [ ]:
df1_train= df_train.copy()
df1_train['cleaned_Text']= ""
df1_test= df_test.copy()
df1_test['cleaned_Text']= ""

In [ ]:
import re
def clean_function(Text):
    # removing the symbols and numbers
    Text = re.sub(r'[\([{})\]!@#$,"%^*?:;~`0-9]', ' ', Text)
    
    # converting the text to lower case
    Text = Text.lower()
    Text = re.sub('http\S+\s*', ' ', Text)  # remove URLs
    Text = re.sub('RT|cc', ' ', Text)  # remove RT and cc
    Text = re.sub('#\S+', '', Text)  # remove hashtags
    Text = re.sub('@\S+', '  ', Text)  # remove mentions
    Text = re.sub('\s+', ' ', Text)  # remove extra whitespace
    
    return Text

In [ ]:
X_train= df1_train["cleaned_Text"]
y_train= df1_train["labels"]
X_test= df1_test["cleaned_Text"]
y_test= df1_test["labels"]

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder= LabelEncoder()

y_train= encoder.fit_transform(y_train)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
CV= CountVectorizer()
X_train= CV.fit_transform(X_train).toarray()
X_train.shape

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")


def get_embeddings(sentences):
  '''return BERT-like embeddings of input text
  Args:
    - sentences: list of strings
  Output:
    - BERT-like embeddings: tf.Tensor of shape=(len(sentences), 768)
  '''
  preprocessed_text = preprocessor(sentences)
  return encoder(preprocessed_text)['pooled_output']

In [ ]:
def get_embeddings(sentences):
    
    preprocessed_text = preprocessor(sentences)
    return encoder(preprocessed_text)['pooled_output']

In [ ]:
import tensorflow as tf
num_classes=17

import numpy as np
X=np.array(df['Message'])
y=np.array(df['Category'])

y = tf.keras.utils.to_categorical(y, num_classes=num_classes)

In [ ]:
i = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
x = preprocessor(i)
x = encoder(x)
x = tf.keras.layers.Dropout(0.2, name="dropout")(x['pooled_output'])
x = tf.keras.layers.Dense(num_classes, activation='softmax', name="output")(x)

model = tf.keras.Model(i, x)

In [ ]:
n_epochs = 20

METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name="accuracy")
]

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                      patience = 3,
                                                      restore_best_weights = True)

model.compile(optimizer = "adam",
              loss = "categorical_crossentropy",
              metrics = METRICS)

model_fit = model.fit(X, 
                      y, 
                      epochs = n_epochs,
                      callbacks = [earlystop_callback])

Epoch 1/20
324/324 [==============================] - 4180s 13s/step - loss: 2.4362 - accuracy: 0.2506


Epoch 2/20
  8/324 [..............................] - ETA: 1:08:48 - loss: 1.6695 - accuracy: 0.5248